In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.swmhau_network_functions import (
    swmhau_network_hyperparameter_search,
)

In [4]:
output_dir = "rumours_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## Rumours

In [5]:
%run load_sbert-embeddings.py

In [6]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


# swmhau Network

In [7]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
include_features_in_path = True
include_features_in_input = True

In [8]:
num_epochs = 100
dimensions = [15]
# define swmhau parameters: (output_channels, sig_depth, num_heads)
swmhau_parameters = [(12, 3, 10), (10, 3, 5)]
num_layers = [1]
ffn_hidden_dim_sizes = [[32,32], [128,128], [512,512]]
dropout_rates = [0.1]
learning_rates = [5e-4, 3e-4, 1e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 3

In [9]:
kwargs = {
    "num_epochs": num_epochs,
    "df": df_rumours,
    "id_column": "timeline_id",
    "label_column": "label",
    "embeddings": sbert_embeddings,
    "y_data": y_data,
    "output_dim": output_dim,
    "dimensions": dimensions,
    "log_signature": True,
    "pooling": "signature",
    "swmhau_parameters": swmhau_parameters,
    "num_layers": num_layers,
    "ffn_hidden_dim_sizes": ffn_hidden_dim_sizes,
    "dropout_rates": dropout_rates,
    "learning_rates": learning_rates,
    "seeds": seeds,
    "loss": loss,
    "gamma": gamma,
    "device": device,
    "features": features,
    "standardise_method": standardise_method,
    "include_features_in_path": include_features_in_path,
    "include_features_in_input": include_features_in_input,
    "split_ids": split_ids,
    "k_fold": True,
    "patience": patience,
    "validation_metric": validation_metric,
    "verbose": False,
}

# w=5

In [10]:
size = 5

## umap

In [11]:
(
    swmhau_network_umap_kfold_5,
    best_swmhau_network_umap_kfold_5,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swmhau_network_umap_focal_2_5_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swmhau_network_umap_focal_2_5_kfold_best_model.csv


In [12]:
swmhau_network_umap_kfold_5.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_2583287/2619527520.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_umap_kfold_5.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         10              3         5         1          (32, 32)       0.1          0.0001         0.687916   
                                                                                      0.0003         0.686735   
                                                                                      0.0005         0.691892   
                                                          (128, 128)     0.1          0.0001         0.691705   
                                                                                      0.0003         0.692824   
                                                                                      0.0005         0.691084   
                                                          (512, 512)     0.1          0.0001         0.690587   
                                                                                      0.0003         0.696738   
                                                                                      0.0005         0.694191   
           12              3         10        1          (32, 32)       0.1          0.0001         0.692078   
                                                                                      0.0003         0.689593   
                                                                                      0.0005         0.682759   
                                                          (128, 128)     0.1          0.0001         0.681578   
                                                                                      0.0003         0.693321   
                                                                                      0.0005         0.695868   
                                                          (512, 512)     0.1          0.0001         0.683939   
                                                                                      0.0003         0.703200   
                                                                                      0.0005         0.702268   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         10              3         5         1          (32, 32)       0.1          0.0001         0.659405   
                                                                                      0.0003         0.656664   
                                                                                      0.0005         0.668137   
                                                          (128, 128)     0.1          0.0001         0.664121   
                                                                                      0.0003         0.662232   
                                                                                      0.0005         0.665875   
                                                          (512, 512)     0.1          0.0001         0.662732   
                                                                                      0.0003         0.667673   
                                                                                      0.0005         0.665958   
           12              3         10        1          (32, 32)       0.1          0.0001         0.659541   
                                                                                      0.0003         0.662144   
                                                                                      0.0005         0.657292   
                                                          (128, 128)     0.1          0.0001         0.657485   
                                                                                      0.0003         0.664921   
                                

In [13]:
best_swmhau_network_umap_kfold_5

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.684809,0.659525,"[0.7523070162589716, 0.5667435306174737]",0.657498,"[0.7690925426774483, 0.5459032576505429]",0.662738,"[0.7362385321100917, 0.5892381459776239]",None,0.742418,...,0.0003,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.695993,0.663289,"[0.7682250959215574, 0.5583536420254535]",0.664677,"[0.7616229923922232, 0.5677312775330396]",0.662112,"[0.7749426605504587, 0.5492807671816728]",None,0.740261,...,0.0003,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.697670,0.663882,"[0.7704500424568356, 0.5573144104803494]",0.666054,"[0.7607602012297373, 0.5713486289871292]",0.662172,"[0.7803899082568807, 0.5439531166755461]",None,0.730961,...,0.0003,123,focal,2,True,5,Conv1d,None,concatenation,64


In [14]:
best_swmhau_network_umap_kfold_5[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

,dimensions,output_channels,sig_depth,num_heads,num_layers,ffn_hidden_dim,dropout_rate,learning_rate
0,15,10,3,5,1,"(128, 128)",0.1,0.0003
0,15,10,3,5,1,"(128, 128)",0.1,0.0003
0,15,10,3,5,1,"(128, 128)",0.1,0.0003


In [15]:
best_swmhau_network_umap_kfold_5["f1"].mean()

0.6622322896267735

In [16]:
best_swmhau_network_umap_kfold_5["precision"].mean()

0.6627431500783535

In [17]:
best_swmhau_network_umap_kfold_5["recall"].mean()

0.6623405217920456

In [18]:
np.stack(best_swmhau_network_umap_kfold_5["f1_scores"]).mean(axis=0)

array([0.76366072, 0.56080386])

In [19]:
np.stack(best_swmhau_network_umap_kfold_5["precision_scores"]).mean(axis=0)

array([0.76382525, 0.56166105])

In [20]:
np.stack(best_swmhau_network_umap_kfold_5["recall_scores"]).mean(axis=0)

array([0.76385703, 0.56082401])

# w=11

In [21]:
size = 11

## umap

In [22]:
(
    swmhau_network_umap_kfold_11,
    best_swmhau_network_umap_kfold_11,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/swmhau_network_umap_focal_2_11_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/swmhau_network_umap_focal_2_11_kfold_best_model.csv


In [23]:
swmhau_network_umap_kfold_11.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_2583287/404807304.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_umap_kfold_11.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         10              3         5         1          (32, 32)       0.1          0.0001         0.683318   
                                                                                      0.0003         0.683442   
                                                                                      0.0005         0.682883   
                                                          (128, 128)     0.1          0.0001         0.678596   
                                                                                      0.0003         0.689220   
                                                                                      0.0005         0.684933   
                                                          (512, 512)     0.1          0.0001         0.684809   
                                                                                      0.0003         0.696738   
                                                                                      0.0005         0.699534   
           12              3         10        1          (32, 32)       0.1          0.0001         0.684188   
                                                                                      0.0003         0.686362   
                                                                                      0.0005         0.691022   
                                                          (128, 128)     0.1          0.0001         0.676111   
                                                                                      0.0003         0.680708   
                                                                                      0.0005         0.693756   
                                                          (512, 512)     0.1          0.0001         0.684250   
                                                                                      0.0003         0.699969   
                                                                                      0.0005         0.696365   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         10              3         5         1          (32, 32)       0.1          0.0001         0.661202   
                                                                                      0.0003         0.659843   
                                                                                      0.0005         0.659256   
                                                          (128, 128)     0.1          0.0001         0.656791   
                                                                                      0.0003         0.667767   
                                                                                      0.0005         0.662831   
                                                          (512, 512)     0.1          0.0001         0.665253   
                                                                                      0.0003         0.669657   
                                                                                      0.0005         0.670433   
           12              3         10        1          (32, 32)       0.1          0.0001         0.663879   
                                                                                      0.0003         0.666376   
                                                                                      0.0005         0.669058   
                                                          (128, 128)     0.1          0.0001         0.653527   
                                                                                      0.0003         0.658907   
                                

In [24]:
best_swmhau_network_umap_kfold_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.682945,0.659349,"[0.749003984063745, 0.5696939033645333]",0.657026,"[0.7716631194892064, 0.5423892100192679]",0.663766,"[0.7276376146788991, 0.5998934469898775]",None,0.770454,...,0.0003,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.685741,0.667179,"[0.7457780458383594, 0.5885797950219619]",0.664788,"[0.7865776081424937, 0.5429986492570914]",0.675758,"[0.7090022935779816, 0.6425146510388918]",None,0.782855,...,0.0003,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.673439,0.650194,"[0.7403675163011263, 0.5600200904068307]",0.647974,"[0.7662576687116565, 0.5296912114014252]",0.655101,"[0.7161697247706422, 0.594033031433138]",None,0.764254,...,0.0003,123,focal,2,True,5,Conv1d,None,concatenation,64


In [25]:
best_swmhau_network_umap_kfold_11[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

,dimensions,output_channels,sig_depth,num_heads,num_layers,ffn_hidden_dim,dropout_rate,learning_rate
0,15,12,3,10,1,"(128, 128)",0.1,0.0003
0,15,12,3,10,1,"(128, 128)",0.1,0.0003
0,15,12,3,10,1,"(128, 128)",0.1,0.0003


In [26]:
best_swmhau_network_umap_kfold_11["f1"].mean()

0.658907222499426

In [27]:
best_swmhau_network_umap_kfold_11["precision"].mean()

0.6565962445035235

In [28]:
best_swmhau_network_umap_kfold_11["recall"].mean()

0.6648751270815717

In [29]:
np.stack(best_swmhau_network_umap_kfold_11["f1_scores"]).mean(axis=0)

array([0.74504985, 0.5727646 ])

In [30]:
np.stack(best_swmhau_network_umap_kfold_11["precision_scores"]).mean(axis=0)

array([0.7748328 , 0.53835969])

In [31]:
np.stack(best_swmhau_network_umap_kfold_11["recall_scores"]).mean(axis=0)

array([0.71760321, 0.61214704])

# w=20

In [32]:
size = 20

## umap

In [ ]:
(
    swmhau_network_umap_kfold_20,
    best_swmhau_network_umap_kfold_20,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
swmhau_network_umap_kfold_20.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_umap_kfold_20

In [ ]:
best_swmhau_network_umap_kfold_20[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

In [ ]:
best_swmhau_network_umap_kfold_20["f1"].mean()

In [ ]:
best_swmhau_network_umap_kfold_20["precision"].mean()

In [ ]:
best_swmhau_network_umap_kfold_20["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_umap_kfold_20["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_20["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_20["recall_scores"]).mean(axis=0)

# w=35

In [ ]:
size = 35

## umap

In [ ]:
(
    swmhau_network_umap_kfold_35,
    best_swmhau_network_umap_kfold_35,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_umap_kfold_35.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_umap_kfold_35

In [ ]:
best_swmhau_network_umap_kfold_35[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

In [ ]:
best_swmhau_network_umap_kfold_35["f1"].mean()

In [ ]:
best_swmhau_network_umap_kfold_35["precision"].mean()

In [ ]:
best_swmhau_network_umap_kfold_35["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_umap_kfold_35["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_35["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_35["recall_scores"]).mean(axis=0)

# w=80

In [ ]:
size = 80

## umap

In [ ]:
(
    swmhau_network_umap_kfold_80,
    best_swmhau_network_umap_kfold_80,
    _,
    __,
) = swmhau_network_hyperparameter_search(
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    **kwargs,
)

In [ ]:
swmhau_network_umap_kfold_80.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_swmhau_network_umap_kfold_80

In [ ]:
best_swmhau_network_umap_kfold_80[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

In [ ]:
best_swmhau_network_umap_kfold_80["f1"].mean()

In [ ]:
best_swmhau_network_umap_kfold_80["precision"].mean()

In [ ]:
best_swmhau_network_umap_kfold_80["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_umap_kfold_80["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_80["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_umap_kfold_80["recall_scores"]).mean(axis=0)